### Workspace

In [ ]:
from azureml.core import Workspace

ws = Workspace.create(name="Class1",
                      subscription_id = "cd97d7c8-07fe-4d38-85b4-4f053b920f5d",
                      resource_group = "__hw1__",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

In [ ]:
################ score.py ###################
scorepy=f"""c
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()